## Creating Graph

In [1]:
class Vertex:
    def __init__(self, name) -> None:
        self._id = name
        self._weight = None
        self._adjacent = {}
#------ setting attributes ------#
    def set_id(self, name):
        self._id = name
    def set_weight(self, weight):
        self._weight = weight
    def add_adjacent(self, other_vertex, edge_weight=None):
        self._adjacent[other_vertex._id] = (other_vertex, edge_weight)
#------  getting attributes ------#
    def get_adjacent_ids(self):
        res = {key:value[1] for key, value in self._adjacent.items()}
        return res

In [2]:
class Graph:
    def __init__(self) -> None:
        self.vertices = {}
#------ setting attributes ------#
    def add_vertex(self, name):
        vertex = Vertex(name)
        self.vertices[name] = vertex
        return vertex
    def add_undirected_edge(self, vertex_a, vertex_b, edge_weight=None):
        vertex_a.add_adjacent(other_vertex = vertex_b, edge_weight=edge_weight)
        vertex_b.add_adjacent(other_vertex = vertex_a, edge_weight=edge_weight)
    def add_directed_edge(self, start, end, edge_weight=None):
        start.add_adjacent(other_vertex = end, edge_weight = edge_weight)
#------ getting attributes ------#
    def get_vertex(self, name):
        return self.vertices[name]

## Depth First Search with Stacks

In [3]:
# DFS return True or False only  
# Every time check the first element in the stack, if it is not the target;
# iterate the adjacent vertices and put them in the stack; 
# repeat above steps, avoid visited vertices.
# Keep it going until there is no way and than back to last intersection.
class Graph_DFS_basic(Graph):
    def get_dfs_basic(self, start_id, stop_id):
        stack = list()
        visited = set()
        
        stack.append(start_id)
        while stack:
            current = stack.pop()
            visited.add(current)
            if current == stop_id:
                return True
            
            for id, vertex in self.get_vertex(current)._adjacent.items():
                if id not in visited:
                    stack.append(id)
        return False

In [4]:
g = Graph_DFS_basic()
va = g.add_vertex('a')
vb = g.add_vertex('b')
vc = g.add_vertex('c')
vd = g.add_vertex('d')
ve = g.add_vertex('e')
vf = g.add_vertex('f')
g.add_undirected_edge(va,vb)
g.add_undirected_edge(vb,vc)
g.add_undirected_edge(vb,vd)
g.add_undirected_edge(vc,vd)
g.add_undirected_edge(vb,ve)
g.add_undirected_edge(vd,ve)
g.get_dfs_basic('a', 'e')

True

In [5]:
# DFS return Path weights and Path route  
# The attribute weight for each vertex represents the sum of last vertex's weight and the edge weight.
# Using dictionary to record each vertex's last vertex, and iterating the dictionary from end to start find the path.
class Graph_DFS_weights(Graph):
    def get_dfs_route(self, start_id, stop_id):
        stack = list()
        visited = set()
        predecessor = {start_id:None}
        
        self.get_vertex(start_id).set_weight(0)
        stack.append(start_id)
        while stack:
            current = stack.pop()
            visited.add(current)
            if current == stop_id:
                path = []
                while current is not None:
                    path.insert(0, current)
                    current = predecessor[current]
                return True, self.get_vertex(stop_id)._weight, path
            
            for id, vertex in self.get_vertex(current)._adjacent.items():
                if id not in visited:
                    vertex[0].set_weight(self.get_vertex(current)._weight + vertex[1])
                    stack.append(id)
                    predecessor[id] = current
        
        return False

In [6]:
g = Graph_DFS_weights()
va = g.add_vertex('a')
vb = g.add_vertex('b')
vc = g.add_vertex('c')
vd = g.add_vertex('d')
ve = g.add_vertex('e')
vf = g.add_vertex('f')
g.add_undirected_edge(va,vb, 1)
g.add_undirected_edge(vb,vc, 2)
g.add_undirected_edge(vb,vd, 1)
g.add_undirected_edge(vc,vd, 4)
g.add_undirected_edge(vb,ve, 3)
g.add_undirected_edge(vd,ve, 5)
g.get_dfs_route('a', 'd')

(True, 9, ['a', 'b', 'e', 'd'])

## Breadth First Search Algorithm  
The property of queue is FIFO (first in first out).  
So, BFS is different from DFS, it using FIFO to go through all vertices on the same level and then move to next level vertices.

In [7]:
# Basic BFS just return True or False
from collections import deque

class Graph_BFS_basic(Graph):
    def get_bfs_basic(self, start, goal):
        visited = set()
        queue = deque()

        visited.add(start)
        queue.append(start)
        while queue:
            current = queue.popleft()
            visited.add(current)
            if current == goal:
                return True
            for id, vertex in self.get_vertex(current)._adjacent.items():
                if id not in visited:
                    queue.append(id)
                    
        return False

In [8]:
g = Graph_BFS_basic()
va = g.add_vertex('a')
vb = g.add_vertex('b')
vc = g.add_vertex('c')
vd = g.add_vertex('d')
ve = g.add_vertex('e')
vf = g.add_vertex('f')
g.add_undirected_edge(va,vb, 1)
g.add_undirected_edge(va,vc, 2)
g.add_undirected_edge(vb,vd, 1)
g.add_undirected_edge(vb,ve, 4)

g.get_bfs_basic('a', 'c')

True

**Note**: 
some times the graph if not sample tree structure but a cyclic structure.  
Although BFS do it well but the return path would be incorrect.  
Pay attention on the existed elements in queue.

In [9]:
# BFS with weight and path
from collections import deque

class Graph_BFS_weight(Graph):
    def get_bfs_weight(self, start, goal):
        visited = set()
        queue = deque()
        predecessor = {start:None}
        self.get_vertex(start).set_weight(0)

        visited.add(start)
        queue.append(start)
        while queue:
            current = queue.popleft()
            visited.add(current)
            if current == goal:
                path = []
                while current is not None:
                    path.insert(0, current)
                    current = predecessor[current]

                return True, path, self.get_vertex(goal)._weight

            for id, vertex in self.get_vertex(current)._adjacent.items():
                if id not in visited and id not in queue:
                    queue.append(id)
                    vertex[0].set_weight(vertex[1]+self.get_vertex(current)._weight)
                    predecessor[id] = current
                    
        return False

In [10]:
g = Graph_BFS_weight()
va = g.add_vertex('a')
vb = g.add_vertex('b')
vc = g.add_vertex('c')
vd = g.add_vertex('d')
ve = g.add_vertex('e')
vf = g.add_vertex('f')
g.add_undirected_edge(va,vb, 1)
g.add_undirected_edge(vb,vc, 3)
g.add_undirected_edge(vb,vd, 2)
g.add_undirected_edge(vb,ve, 1)
g.add_undirected_edge(vc,ve, 4)
g.add_undirected_edge(vc,vd, 1)
g.add_undirected_edge(ve,vf, 3)
g.add_undirected_edge(vd,va, 2)
g.add_undirected_edge(vd,ve, 2)

g.get_bfs_weight('a', 'e')

(True, ['a', 'b', 'e'], 2)

## Dijkstra's Algorithm